# Fine-Tuning Preparation
Based on the analysis, the confidence score is correlated with the number of labels predicted. This means that by increasing the number of predicted labels, the confidence score will increase as wel. However, this would only appply after the prediction.
The good news is that from the analysis, there are a few label types that have shown to have poor high confidence score ratio against low confidence score. This is while some low frequency labels have good ratio. So, to increase the confidence score with less label frequency. The training data needs to be augmented. Here, there are two things that can be done for this augmentation. These are:
- get contextual texts that corresponds to poor ratio labels.
- synthesize training data for rare texts.

------------
-----------
## Data Preparation

### Import Libraries

In [7]:
import pandas as pd
from gliner import GLiNER
from torch.utils.data import DataLoader
import torch

### Import Model

In [8]:
model = GLiNER.from_pretrained("urchade/gliner_multi")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 69905.07it/s]


GLiNER(
  (model): SpanModel(
    (token_rep_layer): Encoder(
      (bert_layer): Transformer(
        (model): DebertaV2Model(
          (embeddings): DebertaV2Embeddings(
            (word_embeddings): Embedding(250105, 768, padding_idx=0)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): DebertaV2Encoder(
            (layer): ModuleList(
              (0-11): 12 x DebertaV2Layer(
                (attention): DebertaV2Attention(
                  (self): DisentangledSelfAttention(
                    (query_proj): Linear(in_features=768, out_features=768, bias=True)
                    (key_proj): Linear(in_features=768, out_features=768, bias=True)
                    (value_proj): Linear(in_features=768, out_features=768, bias=True)
                    (pos_dropout): Dropout(p=0.1, inplace=False)
                    (dropout): Dropout(p=0.1, inplace=False)
            

### Data Preparation

### Data Loader

In [10]:
# 3. Create data loaders using model's built-in collate function
train_loader = DataLoader(
    train_data, 
    batch_size=4, 
    shuffle=True,
    collate_fn=model.collate_fn
)

val_loader = DataLoader(
    val_data,
    batch_size=4,
    collate_fn=model.collate_fn
)

AttributeError: 'GLiNER' object has no attribute 'collate_fn'

### Optimizer

In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=2e-5
)

### Training

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0
    for batch in train_loader:
        # Move batch to device
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v 
                 for k, v in batch.items()}
        
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v 
                     for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()
    
    # Print epoch stats
    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss/len(train_loader):.4f} | "
          f"Val Loss: {val_loss/len(val_loader):.4f}")

### Save Model

In [ ]:
model.save_pretrained("fine_tuned_gliner")